In [1]:
import re
import pandas as pd
from pprint import pprint
import math

In [2]:
private_tags_table = pd.read_csv('../docs/TCIAPrivateTagKB-02-01-2024-formatted.csv')

private_tags_table['vr'] = private_tags_table['vr'].astype('category')
private_tags_table['private_disposition'] = private_tags_table['private_disposition'].astype('category')

In [3]:
private_tags_table

,element_sig_pattern,tag_name,vr,private_disposition
0,"(0008,1084)[<0>](312f,""Ramsoft Diagnosis Datet...",Admitting Diagnoses Code Sequence:Unknown,DA,d
1,"(0008,1084)[<0>](312f,Ramsoft Diagnosis Dateti...",Admitting Diagnoses Code Sequence:Unknown,TM,d
2,"(0008,1084)[<0>](3131,Ramsoft Diagnosis Modifi...",Admitting Diagnoses Code Sequence:Unknown,DA,d
3,"(0008,1084)[<0>](3131,Ramsoft Diagnosis Modifi...",Admitting Diagnoses Code Sequence:Unknown,TM,d
4,"(0008,1084)[<0>](3133,Ramsoft Diagnosis Status...",Admitting Diagnoses Code Sequence:Unknown,IS,d
...,...,...,...,...
8782,"(8053,Unnamed Private Block - 10"",00)",Unknown,OB,d
8783,"(f215,PB group A"",17)",Unknown,DA,d
8784,"(f215,PB group A"",17)",Unknown,UN,d
8785,"(f215,PB group A"",18)",Unknown,TM,d


In [4]:
private_tags_table['private_disposition'].value_counts()

private_disposition
d     4585
k     3811
na     210
h       84
o       46
 k      23
oi       9
D        2
K        2
Name: count, dtype: int64

In [5]:
# private_tag_dict = {}

# def get_tag_n_creator_from_element_str(elemnt_str):
#     cleaned_tag = elemnt_str.strip('(').strip(')')
#     splitted = cleaned_tag.split(',')
    
#     private_creator = 'empty'
#     if len(splitted) == 3 and splitted[1].strip() != '':            
#         private_creator = splitted[1].strip('"')
#     elif len(splitted) > 3:
#         private_creator = ''.join(splitted[1:-1]).strip('"')

#     if len(splitted) == 2:
#         tag = f"({splitted[0]},{splitted[-1]})"
#     else:
#         tag = f"({splitted[0]},{splitted[-1][-2:]})"

#     return tag, private_creator.lower()

# conflict_tags_count = {}

# for index, row in private_tags_table.iterrows():
#     element_pattern = row['element_sig_pattern']
#     element_tags = re.split(r'\[\<\d+\>\]', element_pattern)
    
#     tag, _ = get_tag_n_creator_from_element_str(element_tags[-1])
#     private_creator = 'empty'
#     all_tags = []
#     for eltag in element_tags:
#         ptag, prcreator  = get_tag_n_creator_from_element_str(eltag)
#         all_tags.append(ptag)
#         if private_creator == 'empty' and prcreator != 'empty':
#            private_creator = prcreator
    
#     disposition_val = row['private_disposition']
#     if not isinstance(disposition_val, str):
#         disposition_val = 'k'

#     vr = row['vr']
#     if not isinstance(vr, str):
#         vr = 'UN'

#     disposition_val = disposition_val.strip().lower()
        
#     tag_dict = {
#         'tag': tag,
#         'pattern': element_pattern,
#         'tag_name': row['tag_name'],
#         'vr': vr,
#         'private_disposition': disposition_val
#     }
    
    
#     private_creator_dict = {}
#     if private_creator in private_tag_dict.keys():
#         private_creator_dict = private_tag_dict[private_creator]

#     unique_tag_key = f"{'_'.join(all_tags)}_{row['vr']}"

#     if unique_tag_key in private_creator_dict.keys():
#         if element_pattern.lower() == private_creator_dict[unique_tag_key]['pattern'].lower():
#             pass
#         else:
#             conflict_tags_count[unique_tag_key] = f"{element_pattern} / {private_creator_dict[unique_tag_key]['pattern']}"
#             print(f'WARNING!! {unique_tag_key} already exist in the private creator dict "{private_creator}"')
#             print(element_pattern)
#             print(private_creator_dict[unique_tag_key]['pattern'])
#             print('====================================================')

#     private_creator_dict[unique_tag_key] = tag_dict
#     private_tag_dict[private_creator] = private_creator_dict

#     # if index >= 1000:
#     #     break

# print(len(conflict_tags_count.keys()))

In [19]:
private_tag_dict = {}
conflict_tags_count = {}

for index, row in private_tags_table.iterrows():
    element_pattern = row['element_sig_pattern']
    splitted = element_pattern.split(',')
    splitted = [s.strip('"').strip().lower() for s in splitted]
    pattern_cleaned = ','.join(splitted)

    disposition_val = row['private_disposition']
    if not isinstance(disposition_val, str):
        disposition_val = 'k'

    vr = row['vr']
    if not isinstance(vr, str):
        vr = 'UN'

    disposition_val = disposition_val.strip().lower()
        
    tag_dict = {
        'pattern': element_pattern,
        'tag_name': row['tag_name'],
        'vr': vr,
        'private_disposition': disposition_val
    }

    unique_tag_key = f"{pattern_cleaned}_{row['vr']}"
    
    if unique_tag_key in private_tag_dict.keys():
        if disposition_val == private_tag_dict[unique_tag_key]['private_disposition']:
            pass
        else:
            conflict_tags_count[unique_tag_key] = f"{element_pattern} / {private_tag_dict[unique_tag_key]['pattern']}"
            print(f'WARNING!! {unique_tag_key} already exist in the private_tag_dict as "{element_pattern}"')
            print(f"{private_tag_dict[unique_tag_key]['private_disposition']} -> {disposition_val}")
            print('====================================================')

    private_tag_dict[unique_tag_key] = tag_dict

print(len(conflict_tags_count.keys()))

WARNING!! (2005,philips mr imaging dd 001,18)_LO already exist in the private_tag_dict as "(2005,PHILIPS MR IMAGING DD 001",18)"
k -> d
WARNING!! (2005,philips mr imaging dd 001,35)_CS already exist in the private_tag_dict as "(2005,PHILIPS MR IMAGING DD 001",35)"
d -> k
WARNING!! (2005,philips mr imaging dd 001,35)_SS already exist in the private_tag_dict as "(2005,PHILIPS MR IMAGING DD 001",35)"
d -> k
WARNING!! (2005,philips mr imaging dd 001,5f)_CS already exist in the private_tag_dict as "(2005,PHILIPS MR IMAGING DD 001",5f)"
d -> k
WARNING!! (2005,philips mr imaging dd 001,5f)_SQ already exist in the private_tag_dict as "(2005,PHILIPS MR IMAGING DD 001",5f)"
d -> k
WARNING!! (2005,philips mr imaging dd 001,70)_LO already exist in the private_tag_dict as "(2005,PHILIPS MR IMAGING DD 001",70)"
k -> d
WARNING!! (2005,philips mr imaging dd 001,80)[<0>](2005,philips mr imaging dd 001,40)_SL already exist in the private_tag_dict as "(2005,PHILIPS MR IMAGING DD 001",80)[<0>](2005,"PHILI

In [20]:
# import json

# with open('../docs/tcia_private_tags_dict.json', 'w', encoding='utf-8') as f:
#     json.dump(private_tag_dict, f, ensure_ascii=False, indent=4)

In [23]:
import pydicom
from pydicom.dataset import Dataset, DataElement
from pydicom.sequence import Sequence

from pydicom import dcmread

from pathlib import Path

In [74]:
validation_data_path = "/home/r079a/Desktop/de-identification/dataset/midi-validation-data"
sample_dicom = Path(validation_data_path, 'input_data/2663440963/1.3.984.1.0.3271391.3.552.1032524404977493356/1.3.984.1.0.3271391.3.552.1757728462147515590/00000007.dcm')

# sample_dicom = Path(validation_data_path, 'input_data/2951928526/2.1.846.0.0.0624952.3.426.1171423858789191618/2.1.846.0.0.0624952.3.426.4666769529569841014/00000001.dcm')

print(sample_dicom)

/home/r079a/Desktop/de-identification/dataset/midi-validation-data/input_data/2663440963/1.3.984.1.0.3271391.3.552.1032524404977493356/1.3.984.1.0.3271391.3.552.1757728462147515590/00000007.dcm


In [75]:
dataset = dcmread(sample_dicom)

In [76]:
def extract_private_groups_n_creators(dataset):
    creators = []
    groups = []
    for element in dataset:
        if element.VR == 'OW':
            continue
        if element.tag.is_private:
            groups.append(f"{element.tag.group:04x}")
            if element.name == 'Private Creator' and element.value not in creators:                
                creators.append(element.value)

    groups = list(set(groups))

    return groups, creators

In [77]:
groups, creators = extract_private_groups_n_creators(dataset)

print(groups)
print(creators)

['0027', '0903', '0905', '0009', '0021', '0019', '0043', '0029', '0023', '0025', '7fd1']
['GEMS_IDEN_01', 'GEIIS', 'GEMS_ACQU_01', 'GEMS_RELA_01', 'GEMS_STDY_01', 'GEMS_SERS_01', 'GEMS_IMAG_01', 'GEMS_IMPS_01', 'GEMS_PARM_01', 'GEIIS PACS']


In [78]:
for element in dataset:
    if element.VR == 'OW':
        continue
    if element.tag.is_private:
        print(element)

(0009, 0010) Private Creator                     LO: 'GEMS_IDEN_01'
(0009, 0011) Private Creator                     LO: 'GEIIS'
(0019, 0010) Private Creator                     LO: 'GEMS_ACQU_01'
(0019, 10d9) [Concatenated SAT {# DTI Diffusion  DS: '0.0'
(0021, 0010) Private Creator                     LO: 'GEMS_RELA_01'
(0021, 105a) [Diffusion direction]               SL: 7
(0023, 0010) Private Creator                     LO: 'GEMS_STDY_01'
(0025, 0010) Private Creator                     LO: 'GEMS_SERS_01'
(0027, 0010) Private Creator                     LO: 'GEMS_IMAG_01'
(0029, 0010) Private Creator                     LO: 'GEMS_IMPS_01'
(0043, 0010) Private Creator                     LO: 'GEMS_PARM_01'
(0043, 1039) [Slop_int_6... slop_int_9]          IS: [0, 8, 0, 0]
(0903, 0010) Private Creator                     LO: 'GEIIS PACS'
(0905, 0010) Private Creator                     LO: 'GEIIS'
(7fd1, 0010) Private Creator                     LO: 'GEIIS'


In [79]:
# def element_to_tag_str(element, trimmed_element_tag: bool = True, use_pattern: bool = False):
#     # Convert to hexadecimal string format
#     group_str = f"{element.tag.group:04x}"   # Output: '0010'
#     element_str = f"{element.tag.element:04x}" # Output: '0010'

#     if use_pattern:
#         return f"({group_str},.*{element_str[-2:]})"
#     elif trimmed_element_tag:
#         return f"({group_str},{element_str[-2:]})"
#     else:
#         return f"({group_str},{element_str})"

# def get_dict_key_from_element(element, parent_elements: list):
#     parent_elem_str = ''
#     for elem in parent_elements:
#         parent_elem_str += f"{element_to_tag_str(elem)}_"

#     tag_str = element_to_tag_str(element)
    
#     return f"{parent_elem_str}{tag_str}_{element.VR}"

# def get_key_pattern_from_element(element, parent_elements: list):
#     parent_elem_str = ''
#     for elem in parent_elements:
#         parent_elem_str += f"{element_to_tag_str(elem, use_pattern=True)}_"

#     tag_str = element_to_tag_str(element, use_pattern=True)
    
#     return f"{parent_elem_str}{tag_str}_{element.VR}" 

# def get_private_tags_disposition_from_dict(dict_key: str, dict_pattrn: str = ''):
#     found_dict = None
#     for creator in creators:
#         creator_dict = private_tag_dict[creator.lower()]
#         if dict_key in creator_dict:
#             found_dict = creator_dict[dict_key]
#             break

def get_element_block_tag(element, private_block_name=None):
    group_str = f"{element.tag.group:04x}"   # Output: '0010'
    element_str = f"{element.tag.element:04x}" # Output: '0010'

    if private_block_name is None:
        return f"({group_str},{element_str})"
    else:
        return f"({group_str},{private_block_name.lower()},{element_str[-2:]})"

def get_element_block_tag_with_parents(element, private_block_name=None, parent_blocks: list = []):
    parent_block_str = ""
    for idx, parent_tuple in enumerate(parent_blocks):
        parent_block_tag = get_element_block_tag(parent_tuple[0], parent_tuple[1])
        parent_block_str += f"{parent_block_tag}[<{idx}>]"

    element_block_tag = get_element_block_tag(element, private_block_name)
    return f"{parent_block_str}{element_block_tag}_{element.VR}"

def get_dispoistion_val_from_block_tag(block_tag: str, element: pydicom.DataElement):
    private_rules = private_tag_dict.get(block_tag)
    if private_rules is None:
        if element.name.lower() != "private creator":
            print(f"'{block_tag}' not found")
        return 'k'

    return private_rules['private_disposition']

    
def traverse_dataset(dataset, parent_elements=[], is_root=True):
    private_creator_block_name = None
    for elem in dataset:
        tag = elem.tag
        VR = elem.VR
        value = elem.value
        name = elem.name

        # Check if the root element is private
        if is_root:
            if not elem.tag.is_private:
                continue

        if name == "Private Creator":
            private_creator_block_name = value
        elif len(parent_elements) > 0:
            immidiate_parent = parent_elements[-1]
            private_creator_block_name = immidiate_parent[1]
           
        
        # Process the element
        if isinstance(value, Sequence):
            # If the value is a Sequence, recursively traverse each Dataset in the Sequence
            # print(f"Sequence: {name} (Tag: {tag})")
            updated_parent_elements = parent_elements.copy()
            updated_parent_elements.append((elem, private_creator_block_name))
            for i, item in enumerate(value):
                # print(f"  Item {i} (Parent: {name})")
                traverse_dataset(item, parent_elements=updated_parent_elements, is_root=False)
        else:
            # process the data element
            # dict_key = get_dict_key_from_element(elem, parent_elements)
            # key_pattern = get_key_pattern_from_element(elem, parent_elements)
            block_tag = get_element_block_tag_with_parents(elem, private_creator_block_name, parent_elements)
            private_disposition = get_dispoistion_val_from_block_tag(block_tag, elem)
            print(f"{elem.tag}, {elem.name}, {block_tag}, {private_disposition}")
            
            # get_private_tags_disposition_from_dict(dict_key, key_pattern)
            # patterns = create_pattents_from_element(elem, parent_elements)
            # print(patterns)
            print("------------------")

In [80]:
traverse_dataset(dataset)

(0009, 0010), Private Creator, (0009,gems_iden_01,10)_LO, k
------------------
(0009, 0011), Private Creator, (0009,geiis,11)_LO, k
------------------
(0019, 0010), Private Creator, (0019,gems_acqu_01,10)_LO, k
------------------
(0019, 10d9), [Concatenated SAT {# DTI Diffusion Dir., release 9.0 & below}], (0019,gems_acqu_01,d9)_DS, k
------------------
(0021, 0010), Private Creator, (0021,gems_rela_01,10)_LO, k
------------------
(0021, 105a), [Diffusion direction], (0021,gems_rela_01,5a)_SL, k
------------------
(0023, 0010), Private Creator, (0023,gems_stdy_01,10)_LO, k
------------------
(0025, 0010), Private Creator, (0025,gems_sers_01,10)_LO, k
------------------
(0027, 0010), Private Creator, (0027,gems_imag_01,10)_LO, k
------------------
(0029, 0010), Private Creator, (0029,gems_imps_01,10)_LO, k
------------------
(0043, 0010), Private Creator, (0043,gems_parm_01,10)_LO, k
------------------
(0043, 1039), [Slop_int_6... slop_int_9], (0043,gems_parm_01,39)_IS, k
--------------